In [1]:
import h5py
from model_xray.config import *
from model_xray.path_manager import pm

with h5py.File(pm.get_mcwa_path('famous_le_10m'), 'r') as f:
    for mz_name in f:
        print(mz_name)
        mz = f[mz_name]
        print(mz.dtype, mz.shape)

DenseNet121
float32 (1, 8062504)
EfficientNetV2B0
float32 (1, 7200312)
EfficientNetV2B1
float32 (1, 8212124)
MobileNet
float32 (1, 4253864)
MobileNetV2
float32 (1, 3538984)
MobileNetV3Large
float32 (1, 5507432)
MobileNetV3Small
float32 (1, 2554968)
NASNetMobile
float32 (1, 5326716)


In [3]:
from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import x_lsb_attack, x_lsb_extract

import numpy as np

arr = np.zeros((10, ), dtype=np.uint8)

payload = np.array([0, 1, 2, 3, 4, 5, 6, 7, 20, 14], dtype=np.uint8)
payload_bytes = payload.tobytes()

config = XLSBAttackConfig(
    x = 3,
    fill=True,
    payload_type=PayloadType.PYTHON_BYTES,
    payload_bytes=payload_bytes
)

arr_attacked = x_lsb_attack(arr, config)
print(arr)
print(arr_attacked)

extracted_bytes = x_lsb_extract(arr_attacked, config)

payload_bytes_as_array = np.frombuffer(payload_bytes, dtype=np.uint8)
extracted_bytes_as_array = np.frombuffer(extracted_bytes, dtype=np.uint8)

print(payload_bytes_as_array)
print(extracted_bytes_as_array)

assert extracted_bytes == payload_bytes

capacity: 30, n_b: 8
payload bits: 00000000000000010000001000000011000001000000010100000110000001110001010000001110, len(bits): 80
dupe_amount: 1
len(bits): 80
reshaped: Array('bin3', ['000', '000', '000', '000', '000', '100', '000', '010', '000', '000']), remainder: Array('bin50', ['11000001000000010100000110000001110001010000001110'])
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 4 0 2 0 0]
capacity: 30, n_b: 8
bits: Array('bin3', ['000', '000', '000', '000', '000', '100', '000', '010', '000', '000'])
[ 0  1  2  3  4  5  6  7 20 14]
[0 1 2 0]


AssertionError: 

In [1]:
import math
import numpy as np
from bitstring import Dtype
import itertools
import os

import traceback


from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import _x_lsb_attack_numpy, _x_lsb_attack_bitstring, _x_lsb_attack_numpy_bin, x_lsb_extract, mcwa_to_bytes_arr, bytes_arr_to_mcwa

def check_attack(arr, config, attack_func):
    x = config.x

    config.msb = False
    arr_attacked = attack_func(arr, config)

    config.msb = True

    n_b = arr.dtype.itemsize * 8
    unattacked_bits_amnt = n_b - x

    config.x = unattacked_bits_amnt

    unattacked_bytes_orig = x_lsb_extract(arr, config)
    unattacked_bytes_attacked = x_lsb_extract(arr_attacked, config)

    if not unattacked_bytes_orig == unattacked_bytes_attacked:
        print("orig:", mcwa_to_bytes_arr(arr))
        print("attacked:", mcwa_to_bytes_arr(arr_attacked))

        print("extracted orig:", np.frombuffer(unattacked_bytes_orig, dtype=np.uint8))
        print("extracted attacked:", np.frombuffer(unattacked_bytes_attacked, dtype=np.uint8))

    assert unattacked_bytes_orig == unattacked_bytes_attacked

    config.msb = False
    config.x = x
    extracted_bytes = x_lsb_extract(arr_attacked, config)

    assert extracted_bytes == config.payload_bytes

    return arr_attacked

rng = np.random.default_rng()

dtypes = [np.uint16, np.uint32, np.uint64, np.float32, np.float64]

n_ws = [8, 16, 32, 64, 1_000_000]

config = XLSBAttackConfig(
    x = 3,
    fill=True,
    payload_type=PayloadType.PYTHON_BYTES,
    payload_bytes=None
)

tests_c = 0
try:
    for dtype, n_w in itertools.product(dtypes, n_ws):
        dtype_str = dtype.__name__
        # print(dtype_str)
        n_bits = Dtype(dtype_str).length
        # print(n_bits)

        for x in range(1, n_bits-1):
        # for x in [4, 6, 8,]:
            capacity = x*n_w
            byte_capacity = math.ceil(capacity / 8)

            randbytes = os.urandom(byte_capacity)

            config.x = x
            config.payload_bytes = randbytes

            if np.issubdtype(dtype, np.floating):
                arr = rng.random(size=(n_w, ), dtype=dtype)
            elif np.issubdtype(dtype, np.integer):
                arr = rng.integers(low=0, high = 2**n_bits, size=(n_w,), dtype=dtype)
            else:
                raise ValueError(f"Unknown dtype {dtype}")

            # print(f"dtype: {dtype}, n_w: {n_w}, x: {x}")
            arr_attacked_bitstring = check_attack(arr, config, _x_lsb_attack_numpy_bin)

            if x % 8 == 0:
                arr_attacked_numpy = check_attack(arr, config, _x_lsb_attack_numpy)
                if not np.array_equal(arr_attacked_numpy, arr_attacked_bitstring):
                    # print(f"dtype: {dtype}, n_w: {n_w}, x: {x}")
                    # print("orig: \n", mcwa_to_bytes_arr(arr))
                    # print("numpy: \n", mcwa_to_bytes_arr(arr_attacked_numpy))
                    # print("bitstring: \n", mcwa_to_bytes_arr(arr_attacked_bitstring))
                    pass
                assert np.array_equal(arr_attacked_numpy, arr_attacked_bitstring)
            
            tests_c += 1
except Exception as e:
    print(f"dtype: {dtype}, n_w: {n_w}, x: {x}")
    print(e)
    print(traceback.format_exc())
    pass

print(f"Tests passed: {tests_c}") 


Tests passed: 990


In [5]:
import time
import numpy as np

rng = np.random.default_rng()

n_w = 10_000_000
arr = rng.random(size=(n_w, ), dtype=np.float32)

config = XLSBAttackConfig(
    x = 8,
    fill=True,
    payload_type=PayloadType.RANDOM,
    payload_bytes=None
)

start = time.time()
arr_attacked = _x_lsb_attack_numpy(arr, config)
print("bytes ver took: ", time.time() - start)

start = time.time()
arr_attacked = _x_lsb_attack_numpy_bin(arr, config)
print("bin ver took: ", time.time() - start)

start = time.time()
arr_attacked = _x_lsb_attack_bitstring(arr, config)
print("bitstring ver took: ", time.time() - start)

bytes ver took:  0.17110419273376465
bin ver took:  0.37720441818237305
bitstring ver took:  44.787020206451416


In [3]:
import math
import numpy as np
from bitstring import Dtype
import itertools
import os

from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import _x_lsb_attack_numpy, _x_lsb_attack_bitstring, _x_lsb_attack_numpy_bin, x_lsb_extract, mcwa_to_bytes_arr, bytes_arr_to_mcwa

arr = np.array([0b11111110, 0b11111100, 0b11111000], dtype=np.uint8)

payload = np.array([255,], dtype=np.uint8).tobytes()

config = XLSBAttackConfig(
    x = 2,
    fill=True,
    payload_type=PayloadType.PYTHON_BYTES,
    payload_bytes=payload
)

arr_attacked = _x_lsb_attack_numpy_bin(arr, config)

def ret_binstr_arr(a: np.ndarray):
    return [np.binary_repr(x, 8) for x in a]

print(ret_binstr_arr(arr))
print(ret_binstr_arr(arr_attacked))

['11111110', '11111100', '11111000']
['11111111', '11111111', '11111011']


In [3]:
import numpy as np
from bitstring import BitArray, Array

from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import _x_lsb_attack_numpy, _x_lsb_attack_bitstring, x_lsb_extract, mcwa_to_bytes_arr, bytes_arr_to_mcwa

n_w = 8
n_bits = 16
dtype = np.uint16

rng = np.random.default_rng()
# arr = rng.integers(low=0, high = 2**n_bits, size=(n_w,), dtype=dtype)

arr = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=dtype)

c = Array(str(arr.dtype).lower())
c.fromfile(arr.tobytes())

print(c)

# recon = np.frombuffer(c.tobytes(), dtype=dtype).reshape(arr.shape)

# print("orig: \n", mcwa_to_bytes_arr(arr))
# print("recon: \n", mcwa_to_bytes_arr(recon))

Array('uint16', [256, 512, 768, 1024, 1280, 1536, 1792, 2048])


In [3]:
import numpy as np

n_b = 8
byte_amnt = n_b // 8
dt = np.dtype(f'uint{n_b}')

arr = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=np.uint8)
unpacked = np.unpackbits(arr, bitorder='big').reshape(len(arr), byte_amnt, n_b)
print(unpacked)

[[[0 0 0 0 0 0 0 1]]

 [[0 0 0 0 0 0 1 0]]

 [[0 0 0 0 0 0 1 1]]

 [[0 0 0 0 0 1 0 0]]

 [[0 0 0 0 0 1 0 1]]

 [[0 0 0 0 0 1 1 0]]

 [[0 0 0 0 0 1 1 1]]

 [[0 0 0 0 1 0 0 0]]]


In [24]:
import numpy as np

# n_b = 8
# byte_amnt = n_b // 8
# dt = np.dtype(f'uint{n_b}')

# count = 4

arr = np.array([[1, 2], [100, 200]], dtype=np.uint8)
# arr.resize((len(arr), 1))
arr.resize((*arr.shape,))
unpacked = np.unpackbits(arr, axis=-1, count=6, bitorder='big')
print(unpacked)


addition = np.ones(shape=(2,2), dtype=np.uint8)

print(addition)
stacked = np.hstack([unpacked, addition])
print(stacked)
final = np.packbits(stacked, axis=-1, bitorder='big')

print(final)

[[0 0 0 0 0 0]
 [0 1 1 0 0 1]]
[[1 1]
 [1 1]]
[[0 0 0 0 0 0 1 1]
 [0 1 1 0 0 1 1 1]]
[[  3]
 [103]]


In [1]:
import numpy as np
from model_xray.utils.general_utils import ndarray_to_bytes_arr, bytes_arr_to_ndarray

from model_xray.utils.image_rep_utils import calc_closest_square, _grayscale_lastmbytes

from model_xray.config_classes import ImageRepConfig, ImageType, GrayscaleLastMBytesConfig

cfg = ImageRepConfig(
    image_type=ImageType.GRAYSCALE_LAST_M_BYTES,
    image_rep_config=GrayscaleLastMBytesConfig(
        m=4
    )
)


arr = np.array([[0.98, 0.1, -1.25], [0.123, 0.6, 1.0]], dtype=np.float32)

img = _grayscale_lastmbytes(arr, cfg)
print(img)
print(img.shape)



[[[ 63  61 122 204]
  [191   0 160   0]
  [225 204  72 205]
  [  0   0   0   0]]

 [[ 61  63 251  25]
  [ 63   0 128   0]
  [231 153 109 154]
  [  0   0   0   0]]]
(2, 4, 4)


In [1]:
import numpy as np
from model_xray.utils.general_utils import ndarray_to_bytes_arr, bytes_arr_to_ndarray

from model_xray.utils.image_rep_utils import calc_closest_square

arr = np.array([[0.98, 0.1, -1.25], [0.123, 0.6, 1.0]], dtype=np.float32)

arr_bytes = ndarray_to_bytes_arr(arr)
print("arr_bytes: \n", arr_bytes)

arr_recon = bytes_arr_to_ndarray(arr_bytes, dtype=np.float32)
print("arr_recon: \n", arr_recon)

m = 4

last_byte = arr_bytes[..., -m:]

n_models, n_weights = arr.shape
print("n_models: ", n_models)
print("n_weights: ", n_weights)

closest_square = calc_closest_square(n_weights)
closest_square_sqrt = int(np.sqrt(closest_square))
print("closest square: ", closest_square)

last_bytes_padded = np.pad(last_byte, ((0, 0), (0, closest_square-n_weights), (0, 0)), mode='constant', constant_values=0)

last_bytes_padded = np.swapaxes(last_bytes_padded, -1, -2)

print("last bytes: \n", last_bytes_padded)

arr_bytes: 
 [[[ 63 122 225  72]
  [ 61 204 204 205]
  [191 160   0   0]]

 [[ 61 251 231 109]
  [ 63  25 153 154]
  [ 63 128   0   0]]]
arr_recon: 
 [[ 0.98   0.1   -1.25 ]
 [ 0.123  0.6    1.   ]]
n_models:  2
n_weights:  3
closest square:  4
last bytes: 
 [[[ 63  61 191   0]
  [122 204 160   0]
  [225 204   0   0]
  [ 72 205   0   0]]

 [[ 61  63  63   0]
  [251  25 128   0]
  [231 153   0   0]
  [109 154   0   0]]]


In [2]:
last_bytes_padded_reshaped = last_bytes_padded.reshape(last_bytes_padded.shape[:-1] + (closest_square_sqrt, closest_square_sqrt))

print(last_bytes_padded_reshaped.shape)
# n_models, n_weights, m

print(last_bytes_padded_reshaped)

(2, 4, 2, 2)
[[[[ 63  61]
   [191   0]]

  [[122 204]
   [160   0]]

  [[225 204]
   [  0   0]]

  [[ 72 205]
   [  0   0]]]


 [[[ 61  63]
   [ 63   0]]

  [[251  25]
   [128   0]]

  [[231 153]
   [  0   0]]

  [[109 154]
   [  0   0]]]]


In [5]:
b = np.block(last_bytes_padded_reshaped)
print(b)
print(b.shape)

# np.dstack(last_bytes_padded_reshaped)

[[[[ 72 205]
   [  0   0]]]


 [[[109 154]
   [  0   0]]]]
(2, 1, 2, 2)


In [9]:
s = np.stack(last_bytes_padded_reshaped, axis=2)
print(s)
print(s.shape)

[[[[ 72 205]
   [109 154]]

  [[  0   0]
   [  0   0]]]]
(1, 2, 2, 2)


In [3]:
def reshape_array(arr):
    n, m, d, _ = arr.shape
    s = int(np.sqrt(m))
    
    # Reshape to (n, s, s, d, d)
    reshaped = arr.reshape(n, s, s, d, d)
    
    # Transpose to (n, s, d, s, d)
    transposed = reshaped.transpose(0, 1, 3, 2, 4)
    
    # Reshape to final shape (n, d*s, d*s)
    result = transposed.reshape(n, d*s, d*s)
    
    return result

b = reshape_array(last_bytes_padded_reshaped)
print(b)
print(b.shape)

[[[ 63  61 122 204]
  [191   0 160   0]
  [225 204  72 205]
  [  0   0   0   0]]

 [[ 61  63 251  25]
  [ 63   0 128   0]
  [231 153 109 154]
  [  0   0   0   0]]]
(2, 4, 4)


In [4]:
import numpy as np

a = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=np.uint8)
a.shape

(8,)

In [5]:
a.ndim

1

In [6]:
a = a.reshape((1, -1))
a.shape

(1, 8)

In [1]:
import numpy as np

from model_xray.utils.general_utils import ndarray_to_bytes_arr, bytes_arr_to_ndarray

dt_initial = np.dtype(np.uint8)
dt_initial = dt_initial.newbyteorder('>')

expected_end_result = np.array([[
    [[61,]], [[251,]],
    [[231,]], [[109,]]
]],
dtype=dt_initial)

intermediate_result = np.array([[
    61, 251, 231, 109
]],
dtype=dt_initial)

dt = np.dtype(np.float32)
dt = dt.newbyteorder('>')

final_arr = np.frombuffer(expected_end_result.tobytes(), dtype=dt)
final_arr
# intermediate_result.shape

array([0.123], dtype='>f4')

In [2]:
a = np.array([0.123, 2.13, 77.0], dtype=np.float32)
a_bytes = ndarray_to_bytes_arr(a)

a_bytes

array([[ 61, 251, 231, 109],
       [ 64,   8,  81, 236],
       [ 66, 154,   0,   0]], dtype=uint8)

In [4]:
from model_xray.utils.image_rep_utils import _grayscale_fourpart


res = _grayscale_fourpart(a)
res

array([[[ 61,  64, 251,   8],
        [ 66,   0, 154,   0],
        [231,  81, 109, 236],
        [  0,   0,   0,   0]]], dtype=uint8)

In [4]:
import keras

import numpy as np
import tensorflow as tf

from model_xray.utils.model_utils import extract_weights

def custom_weight_init(shape, dtype=None):
    return tf.reshape((tf.range(1, np.prod(shape)+1, dtype=dtype) * 0.111 ), shape=shape)

model = keras.Sequential(
        [
            keras.layers.Dense(4, kernel_initializer=custom_weight_init, activation="relu", name="layer1"),
            keras.layers.Dense(17, kernel_initializer=custom_weight_init, activation="relu", name="layer2"),
            # keras.layers.Dense(4, name="layer3"),
        ]
    )



model.build((18, 20))
# x = tf.ones((1, 2))
# y = model(x)

shapes = [w.shape for w in model.weights]
shapes

# w = extract_weights(model)
# w

[TensorShape([20, 4]),
 TensorShape([4]),
 TensorShape([4, 17]),
 TensorShape([17])]

In [5]:
mw = model.weights
mw_k0 = mw[0]
mw_b0 = mw[1]
mw_k1 = mw[2]
mw_b0 = mw[3]

mw_k0 = [[1, 2], [3, 4]]

model.weights

[<tf.Variable 'layer1/kernel:0' shape=(2, 2) dtype=float32, numpy=
 array([[ 1.2158784 ,  1.1422704 ],
        [-0.35471785, -0.77634585]], dtype=float32)>,
 <tf.Variable 'layer1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'layer2/kernel:0' shape=(2, 2) dtype=float32, numpy=
 array([[-0.83238983, -1.0574483 ],
        [ 0.7510748 , -0.59170717]], dtype=float32)>,
 <tf.Variable 'layer2/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [2]:
import numpy as np
from sklearn.model_selection import KFold
n, d = 20,2
X = np.zeros((n,d))
y = np.zeros((n,))

k = 10

kf = KFold(n_splits=k, random_state=None, shuffle=False)

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={X[train_index]}")
    print(f"  Test:  index={X[test_index]}")

Fold 0:
  Train: index=[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
  Test:  index=[[0. 0.]
 [0. 0.]]
Fold 1:
  Train: index=[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
  Test:  index=[[0. 0.]
 [0. 0.]]
Fold 2:
  Train: index=[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
  Test:  index=[[0. 0.]
 [0. 0.]]
Fold 3:
  Train: index=[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
  Test:  index=[[0. 0.]
 [0. 0.]]
Fold 4:
  Train: index=[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.

# Image reps

In [1]:
import numpy as np
from model_xray.config_classes import *
from model_xray.utils.image_rep_utils import _grayscale_weighted_avg

dt_uint8_be = np.dtype('>u1')
dt_float32_be = np.dtype('>f4')

dt_float32_ne = np.dtype('=f4')

expected_end_result = np.array([[
    [50,]
]],
dtype=dt_uint8_be)

a_input = np.array([[0.123,]], dtype=dt_float32_ne)

result = _grayscale_weighted_avg(a_input, config=ImageRepConfig(
    image_type=ImageType.GRAYSCALE_WEIGHTED_AVG,
    image_rep_config=GrayscaleWeightedAvgConfig(),
))
result

2024-08-20 21:55:04.183270: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-20 21:55:04.206779: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 21:55:04.206797: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 21:55:04.206812: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-20 21:55:04.211050: I tensorflow/core/platform/cpu_feature_g

data_bytes:
 [[[ 61 251 231 109]]]
first_2_bytes_unpacked:
 [[[0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1]]]
concated_bytes:
 [[[123]]]
all_bytes:
 [[[123 231 109]]]


array([[[148]]], dtype=uint8)

In [5]:
a_input = np.array([[0.123, 2.13],[77.0, -1.23]], dtype=dt_float32_ne)
result = _grayscale_weighted_avg(a_input, config=ImageRepConfig(
    image_type=ImageType.GRAYSCALE_WEIGHTED_AVG,
    image_rep_config=GrayscaleWeightedAvgConfig(),
))
result

data_bytes:
 [[[ 61 251 231 109]
  [ 64   8  81 236]]

 [[ 66 154   0   0]
  [191 157 112 164]]]
first_2_bytes_unpacked:
 [[[0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1]
  [0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]

 [[0 1 0 0 0 0 1 0 1 0 0 1 1 0 1 0]
  [1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1]]]
concated_bytes:
 [[[123]
  [  8]]

 [[ 26]
  [157]]]
all_bytes:
 [[[123 231 109]
  [  8  81 236]]

 [[ 26   0   0]
  [157 112 164]]]


array([[[148, 143],
        [  0,   0]],

       [[  5, 147],
        [  0,   0]]], dtype=uint8)